# Modelo Preditivo - Competição "Talking Data"
## Alunos: Alessandro Rivello e Raul Guarini

Tentativa de usar um modelo com blacklists de features, tal qual a EDA sugeriu ser promissor, e usando dados de timestamp.

In [14]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Definindo o estilo dos plots
sns.set_style("whitegrid")
sns.set(font_scale=1.5)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

path_raul = '/Users/Raul/Desktop/'

In [42]:
# Definir os datatypes antes da importação otimiza o uso de memória pelo Pandas.
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'category',
        'device'        : 'category',
        'os'            : 'category',
        'channel'       : 'category',
        'is_attributed' : 'uint8',
        'days'          : 'category',
        'hours'         : 'category',
        'minutes'       : 'category',
        'seconds'       : 'category',
        }

In [43]:
# Forçando o Pandas a ler as mesmas colunas, sem necessidade de ler click_time, click_id e days, uma vez que os dados
# de teste e treinamento são de dias diferentes. Logo essa feature não é informativa.

cols_train = ['ip', 'app', 'device', 'os', 'channel', 'is_attributed', 'hours', 'minutes', 'seconds']
cols_test = ['ip', 'app', 'device', 'os', 'channel', 'hours', 'minutes', 'seconds']

In [44]:
train = pd.read_csv(path_raul + '10_train_timed.csv', dtype = dtypes, usecols = cols_train)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18490388 entries, 0 to 18490387
Data columns (total 9 columns):
ip               uint32
app              category
device           category
os               category
channel          category
is_attributed    uint8
hours            category
minutes          category
seconds          category
dtypes: category(7), uint32(1), uint8(1)
memory usage: 282.3 MB


Vamos fazer one-hot-encoding das horas e minutos

In [52]:
train.head()

,ip,app,device,os,channel,is_attributed,hours,minutes,seconds
0,103022,3,1,23,379,0,14,37,44
1,114221,3,1,19,379,0,14,37,59
2,47902,3,1,17,379,0,14,48,7
3,23550,3,1,13,379,0,14,53,39
4,84644,3,1,19,379,0,14,56,49


In [45]:
# Instanciando o Binarizer
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
mlb = MultiLabelBinarizer()